# Setup and Imports

In [1]:
# Essential Imports
import pandas as pd
import numpy as np
import os
import zipfile
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import ipeadatapy as ip
from sklearn.impute import KNNImputer

In [2]:
# Install Kaggle and Ipeadata libraries if they haven't been installed in this session
# Uncomment the line below if running in a fresh environment
#%pip install kaggle ipeadatapy

In [3]:
## talk abou kaggle tokens

In [4]:
# NOTE: If your Drive is already mounted via Colab's interface. If it is not, uncomment the line below.
#from google.colab import drive
# We skip the drive.mount() command here to avoid the Value Error.

# Define the base path to your project folder on Google Drive
# ATTENTION: Verify that this path leads to the folder containing 'dados_macro_brasil.csv'!
#DRIVE_BASE_PATH = '/content/drive/MyDrive/Project_01' # <-- Adjust if your path is different!
#MACRO_DATA_PATH = os.path.join(DRIVE_BASE_PATH, 'brasil_macro_data.csv')

#print("Setup complete. Drive path defined at:", DRIVE_BASE_PATH)

# Block 1: Kaggle Data Download and Unzip (Home Credit Default Risk)

In [5]:
# To download kaggle data via API it is necessary to configure your Token in kaggle profile. 
# After configuring your credentials run the code below in the terminal

#mkdir -p ~/.kaggle
## CRITICAL: Replace the entire string below with your key content
#echo '{"username":"YOUR_USERNAME","key":"YOUR_LONG_API_KEY"}' > ~/.kaggle/kaggle.json
#chmod 600 ~/.kaggle/kaggle.json

In [6]:
DRIVE_BASE_PATH = '/content/drive/MyDrive/Project_01'

# CRITICAL CORRECTION: Use the competition slug and command
KAGGLE_COMPETITION_SLUG = 'home-credit-default-risk' 

# Path Variables (Ensure DRIVE_BASE_PATH is correct, e.g., '/content/drive/MyDrive/Project_01')
KAGGLE_DIR = os.path.join(DRIVE_BASE_PATH, 'data/raw/kaggle') 
DATA_DIR_RAW = os.path.join(KAGGLE_DIR, KAGGLE_COMPETITION_SLUG)
ZIP_FILE = os.path.join(KAGGLE_DIR, KAGGLE_COMPETITION_SLUG + '.zip')

# 1. Create necessary directories
os.makedirs(KAGGLE_DIR, exist_ok=True)
os.makedirs(DATA_DIR_RAW, exist_ok=True)
print(f"Directories created/checked: {KAGGLE_DIR}")

# 2. Download the dataset using the Kaggle API 
print(f"Attempting to download competition dataset: {KAGGLE_COMPETITION_SLUG}")
try:
    # Use 'competitions download -c' syntax
    !kaggle competitions download -c {KAGGLE_COMPETITION_SLUG} -p {KAGGLE_DIR} --force
    print("✅ Kaggle download command executed.")

except Exception as e:
    print(f"❌ ERROR: Kaggle download command failed unexpectedly. Error: {e}")
    # Note: If the 403 error persists, the problem is your API key/rules acceptance, not the command syntax.


# 3. Unzip the downloaded file
print(f"\nUnzipping data from: {ZIP_FILE}")
try:
    with zipfile.ZipFile(ZIP_FILE, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR_RAW)
    print(f"✅ Data unzipped to: {DATA_DIR_RAW}")
    
    # Clean up the zip file
    os.remove(ZIP_FILE)
    print("Zip file removed.")

except FileNotFoundError:
    print(f"❌ ERROR: Zip file not found at {ZIP_FILE}. This means the download failed. Check API key/rules acceptance.")
except Exception as e:
    print(f"❌ ERROR during unzipping: {e}")

Directories created/checked: /content/drive/MyDrive/Project_01/data/raw/kaggle
Attempting to download competition dataset: home-credit-default-risk


100% 687M/688M [00:07<00:00, 79.1MB/s]
100% 688M/688M [00:07<00:00, 90.7MB/s]
✅ Kaggle download command executed.

Unzipping data from: /content/drive/MyDrive/Project_01/data/raw/kaggle/home-credit-default-risk.zip
✅ Data unzipped to: /content/drive/MyDrive/Project_01/data/raw/kaggle/home-credit-default-risk
Zip file removed.


# Block 2: Query and save Brazil Central Bank inflation and interest rates

In [7]:
# Generate and Save Placeholder CSV

# 1. Define the directory path (ensure the folder exists in your Google Drive)

MACRO_DATA_PATH = os.path.join(DRIVE_BASE_PATH, 'brasil_macro_data.csv')

# 2. Create the directory
os.makedirs(DRIVE_BASE_PATH, exist_ok=True)
print(f"Directory check complete: {DRIVE_BASE_PATH}")

# 3. Create a simple, empty DataFrame with the expected columns/index for now
placeholder_data = {
    'date': pd.to_datetime(['2023-01', '2023-02']),
    'SELIC': [13.75, 13.65], 
    'IPCA': [5.79, 5.60] 
}
df_placeholder = pd.DataFrame(placeholder_data)
df_placeholder.set_index('date', inplace=True)

# 4. Save the placeholder DataFrame to the expected path
df_placeholder.to_csv(MACRO_DATA_PATH)

print(f"✅ Placeholder CSV created and saved at: {MACRO_DATA_PATH}")
print("You can now safely run the loading block (Block 2).")

Directory check complete: /content/drive/MyDrive/Project_01
✅ Placeholder CSV created and saved at: /content/drive/MyDrive/Project_01/brasil_macro_data.csv
You can now safely run the loading block (Block 2).


In [8]:
# Constant for the BCB API limitation
BCB_WINDOW_YEARS = 10

In [9]:
# Query the Brazil Central Bank API

def query_bc(serie_code, START_DATE_STR, END_DATE_STR):
    """
    Auxiliary function to query the BCB API directly via URL for a specific time chunk.
    Dates must be in DD/MM/YYYY format as required by the BCB API.
    """
    url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{serie_code}/dados?formato=json&dataInicial={START_DATE_STR}&dataFinal={END_DATE_STR}'
    
    try:
        # BCB returns 'data' and 'valor' columns
        df = pd.read_json(url)
        
        if df.empty:
             print(f"Warning: BCB returned empty data for series {serie_code} from {START_DATE_STR} to {END_DATE_STR}.")
             return pd.DataFrame()
             
        df['date'] = pd.to_datetime(df['data'], dayfirst=True)
        df = df.rename(columns={'valor': str(serie_code)})
        df.set_index('date', inplace=True)
        df = df[[str(serie_code)]] 
        return df
    except Exception as e:
        print(f"❌ Error querying BCB series {serie_code} from {START_DATE_STR} to {END_DATE_STR}: {e}")
        return pd.DataFrame()

In [10]:
# Create Function that query the data from Brazil Central Bank API, with a 10 year windown condition for querying

def get_series_with_pagination(serie_code, global_start_date_str, global_end_date_str):
    """
    Breaks the total time period into 10-year chunks and calls the BCB API for each chunk.
    This bypasses the API's 10-year limit. Input dates must be in DD/MM/YYYY format.
    """
    # Ensure all dates are the FIRST day of their respective month for safer querying
    global_start_date = datetime.strptime(global_start_date_str, '%d/%m/%Y').replace(day=1)
    global_end_date = datetime.strptime(global_end_date_str, '%d/%m/%Y').replace(day=1)
    
    all_data = []
    current_end_date = global_end_date
    
    print(f"Starting pagination for series {serie_code}. Total range: {global_start_date_str} to {global_end_date_str}")
    
    while current_end_date >= global_start_date:
        # Calculate start date 10 years earlier using relativedelta
        current_start_date = current_end_date - relativedelta(years=BCB_WINDOW_YEARS)
        
        # Ensure that the start date of the chunk is the 1st day of the month
        current_start_date = current_start_date.replace(day=1) 
        
        # Ensure the chunk doesn't start before the user's global start date
        if current_start_date < global_start_date:
            current_start_date = global_start_date
            
        # Format dates for the query (DD/MM/YYYY)
        query_start_date_str = current_start_date.strftime('%d/%m/%Y')
        query_end_date_str = current_end_date.strftime('%d/%m/%Y')
        
        print(f"  -> Querying chunk: {query_start_date_str} to {query_end_date_str}")
        
        df_chunk = query_bc(serie_code, query_start_date_str, query_end_date_str)
        
        if not df_chunk.empty:
            all_data.append(df_chunk)
        
        # Update the end date for the next iteration: move back one day from the start date of the current chunk
        current_end_date = current_start_date - relativedelta(days=1)
        
        # Stop the loop if we've processed the earliest required data block
        if current_start_date == global_start_date:
            break

    if not all_data:
        print(f"The query returned no data for series {serie_code}.")
        return pd.DataFrame()
        
    # Concatenate all DataFrames, sort by date, and remove any duplicates
    df_final = pd.concat(all_data).sort_index()
    df_final = df_final[~df_final.index.duplicated(keep='first')]
    print(f"Pagination successful. Total data points: {len(df_final)}")
    return df_final

In [19]:
# Main Macro Data Acquisition (with Pagination)

# --- USER INPUT ---
# Define the global start and end date for the macro data acquisition (DD/MM/YYYY format)
GLOBAL_START_DATE_STR = '01/01/2014' 
GLOBAL_END_DATE_STR = '01/11/2025'   
# ------------------

# 1. Acquire SELIC Rate (BCB Series 432) using Pagination
print("Acquiring SELIC data...")
try:
    # Use the pagination function to bypass the 10-year limit
    df_selic = get_series_with_pagination(
        432, 
        GLOBAL_START_DATE_STR, 
        GLOBAL_END_DATE_STR
    )
    df_selic = df_selic.resample('ME').last()
    df_selic.index = df_selic.index.to_period('M') 
    df_selic.rename(columns={'432': 'SELIC'}, inplace=True)
    print("✅ SELIC Data acquired and paginated successfully.")
    print(f"SELIC rows after resampling: {len(df_selic)}")
except Exception as e:
    print(f"❌ ERROR processing SELIC data: {e}")
    df_selic = pd.DataFrame() 

Acquiring SELIC data...
Starting pagination for series 432. Total range: 01/01/2014 to 01/11/2025
  -> Querying chunk: 01/11/2015 to 01/11/2025
  -> Querying chunk: 01/01/2014 to 31/10/2015
Pagination successful. Total data points: 4323
✅ SELIC Data acquired and paginated successfully.
SELIC rows after resampling: 143


In [12]:
# Acquire IPCA (Brazilian Inflation Index - Ipeadata)
print("\nAcquiring IPCA data...")
try:
    # Convert the end date string for the IPCA year filter
    global_end_date_dt = datetime.strptime(GLOBAL_END_DATE_STR, '%d/%m/%Y')
    
    # IPCA Code GMEC12_IPCA12 (12-month accumulated)
    df_ipca = get_series_with_pagination(
        433, 
        GLOBAL_START_DATE_STR, 
        GLOBAL_END_DATE_STR
    )
    df_ipca.index = df_ipca.index.to_period('M') 
    df_ipca.rename(columns={'433': 'IPCA'}, inplace=True)
    print("✅ IPCA Data acquired successfully.")
except Exception as e:
    print(f"❌ Error acquiring IPCA data from Ipeadata: {e}")
    df_ipca = pd.DataFrame()


Acquiring IPCA data...
Starting pagination for series 433. Total range: 01/01/2014 to 01/11/2025
  -> Querying chunk: 01/11/2015 to 01/11/2025
  -> Querying chunk: 01/01/2014 to 31/10/2015
Pagination successful. Total data points: 142
✅ IPCA Data acquired successfully.


In [13]:
# Merge the datasets, clean, and save
if not df_selic.empty and not df_ipca.empty:
    df_macro = df_ipca.join(df_selic, how='outer') 
    df_macro = df_macro.ffill()
    
    # Save the data to Google Drive
    df_macro.to_csv(MACRO_DATA_PATH)
    
    print("\n✅ Final Macro Data Merged and Saved:")
    print(df_macro.head())
    print(f"Macro Data Shape: {df_macro.shape}")
else:
    print("\n❌ Could not merge data due to empty SELIC or IPCA datasets.")


✅ Final Macro Data Merged and Saved:
         IPCA  SELIC
date                
2014-01  0.55  10.50
2014-02  0.69  10.75
2014-03  0.92  10.75
2014-04  0.67  11.00
2014-05  0.46  11.00
Macro Data Shape: (142, 2)


# Simulate time variable for Credit Dataset

In [14]:
# Define the file path for the main training data
RAW_TRAIN_FILE = os.path.join(DATA_DIR_RAW, 'application_train.csv')

print(f"Attempting to load Micro Data from: {RAW_TRAIN_FILE}")

try:
    df_train = pd.read_csv(RAW_TRAIN_FILE)
    
    # 1. Simulate the Time Variable (Crucial for Macro Merge)
    # The Home Credit data is cross-sectional (no monthly date). 
    # We must assign a random monthly period to each client 
    # for the purpose of joining with the monthly macro data (df_macro).
    
    # Range of dates for simulation (e.g., last 5 years relative to the data creation)
    # The actual date of the dataset is not critical, only the relative time index.
    start_date = datetime(2013, 1, 1)
    end_date = datetime(2018, 5, 1) # Kaggle data was originally published around this time
    
    date_range = pd.period_range(start=start_date, end=end_date, freq='M')
    
    # Assign a random date from the range to each client
    df_train['TIME_INDEX'] = random.choices(date_range, k=len(df_train))
    df_train['TIME_INDEX'] = df_train['TIME_INDEX'].astype('object')
    
    # 2. Convert to Period Index for Joining
    df_train['TIME_INDEX'] = df_train['TIME_INDEX'].apply(lambda x: pd.Period(x, freq='M'))
    
    print("✅ Micro Data loaded and Time Index simulated successfully.")
    print(df_train[['SK_ID_CURR', 'TARGET', 'TIME_INDEX']].head())
    print("\nMicro Data Shape:", df_train.shape)

except FileNotFoundError:
    print(f"❌ ERROR: File not found at {RAW_TRAIN_FILE}. The Kaggle download or unzipping may have failed.")
except Exception as e:
    print(f"❌ An error occurred during micro data loading: {e}")

Attempting to load Micro Data from: /content/drive/MyDrive/Project_01/data/raw/kaggle/home-credit-default-risk/application_train.csv
✅ Micro Data loaded and Time Index simulated successfully.
   SK_ID_CURR  TARGET TIME_INDEX
0      100002       1    2013-12
1      100003       0    2013-06
2      100004       0    2015-02
3      100006       0    2014-03
4      100007       0    2015-02

Micro Data Shape: (307511, 123)


# Load Macro Data and Prepare Time Variable

In [15]:
# The path to the macro data file was defined in Block 1 (e.g., .../Project_01/brasil_macro_data.csv)
print(f"Attempting to load Macro Data from: {MACRO_DATA_PATH}")

try:
    df_macro = pd.read_csv(MACRO_DATA_PATH)
    
    # 1. Convert the Date Column to Datetime
    # We assume the time column is named 'DATE' (or 'date', depending on the API output/saving convention)
    # Adjust 'DATE' if your CSV uses a different column name (e.g., 'date')
    if 'DATE' in df_macro.columns:
        df_macro['DATE'] = pd.to_datetime(df_macro['DATE'])
    elif 'date' in df_macro.columns:
        df_macro['DATE'] = pd.to_datetime(df_macro['date'])
        df_macro = df_macro.drop(columns=['date']) # Clean up the old column if it was renamed
    else:
        raise ValueError("Time column ('DATE' or 'date') not found in macro data.")


    # 2. Convert to Period Index (Monthly Frequency - 'M')
    # This conversion is fundamental for merging with df_train['TIME_INDEX']
    df_macro['TIME_INDEX'] = df_macro['DATE'].dt.to_period('M')
    
    # 3. Drop the original datetime column
    df_macro = df_macro.drop(columns=['DATE'])
    
    # Set TIME_INDEX as the index for easier joining (optional, but good practice)
    df_macro = df_macro.set_index('TIME_INDEX')

    # Display the result
    print("✅ Macro Data loaded and Time Index prepared successfully.")
    print(df_macro.head())
    print("\nMacro Data Shape:", df_macro.shape)

except FileNotFoundError:
    print(f"❌ ERROR: Macro file not found at {MACRO_DATA_PATH}. Check if the file is in your Google Drive.")
except Exception as e:
    print(f"❌ An error occurred during macro data loading or processing: {e}")

Attempting to load Macro Data from: /content/drive/MyDrive/Project_01/brasil_macro_data.csv
✅ Macro Data loaded and Time Index prepared successfully.
            IPCA  SELIC
TIME_INDEX             
2014-01     0.55  10.50
2014-02     0.69  10.75
2014-03     0.92  10.75
2014-04     0.67  11.00
2014-05     0.46  11.00

Macro Data Shape: (142, 2)


## Merging Macro and Micro home credit loan data

In [16]:
# Merge Micro and Macro Data df_train

print("Starting merge of micro (df_train) and macro (df_macro) data...")

# The merge key is the TIME_INDEX (Period('M'))
# We use a LEFT JOIN to keep all client rows from df_train and pull in the macro data.
try:
    # Ensure df_macro is set up for joining by index.
    # The previous steps should have set df_macro.index to TIME_INDEX (Period('M')).
    df_train_enriched = df_train.merge(
        df_macro, 
        left_on='TIME_INDEX', 
        right_index=True,  # Merge df_macro based on its index (TIME_INDEX)
        how='left'
    )
    
    # Check for successful join
    if df_train_enriched.isnull().any().any():
        # Check if only the new macro columns have NaNs (expected for early dates)
        new_cols = df_macro.columns
        if df_train_enriched[new_cols].isnull().all().all():
            print("⚠️ Warning: New macro columns contain NaNs. This is expected if the IPCA data did not cover the full range of TIME_INDEX simulation (e.g., prior to 2000).")
            
    print("✅ Datasets merged successfully.")
    print("\nEnriched Data Sample (with new macro features):")
    # Displaying the target and the newly merged macro features
    print(df_train_enriched[['SK_ID_CURR', 'TARGET', 'TIME_INDEX', 'IPCA', 'SELIC']].head())
    print(f"\nFinal Enriched Shape: {df_train_enriched.shape}")

except Exception as e:
    print(f"❌ An error occurred during the merge process: {e}")

Starting merge of micro (df_train) and macro (df_macro) data...
✅ Datasets merged successfully.

Enriched Data Sample (with new macro features):
   SK_ID_CURR  TARGET TIME_INDEX  IPCA  SELIC
0      100002       1    2013-12   NaN    NaN
1      100003       0    2013-06   NaN    NaN
2      100004       0    2015-02  1.22  12.25
3      100006       0    2014-03  0.92  10.75
4      100007       0    2015-02  1.22  12.25

Final Enriched Shape: (307511, 125)


In [17]:
# Load and Enrich the Test Data (application_test.csv)

# Define the file path for the main test data
RAW_TEST_FILE = os.path.join(DATA_DIR_RAW, 'application_test.csv')

print(f"Attempting to load Test Data from: {RAW_TEST_FILE}")

try:
    df_test = pd.read_csv(RAW_TEST_FILE)
    
    # 1. Simulate the Time Variable (CRITICAL: Use the same date range as df_train)
    
    # Define the range of dates used in Block 5 (e.g., 2013-01-01 to 2018-05-01)
    # Ensure this matches the simulation period used for df_train!
    start_date_sim = datetime(2013, 1, 1)
    end_date_sim = datetime(2018, 5, 1) 
    date_range = pd.period_range(start=start_date_sim, end=end_date_sim, freq='M')
    
    # Assign a random date from the range to each client
    df_test['TIME_INDEX'] = random.choices(date_range, k=len(df_test))
    df_test['TIME_INDEX'] = df_test['TIME_INDEX'].astype('object')
    
    # Convert to Period Index for Joining
    df_test['TIME_INDEX'] = df_test['TIME_INDEX'].apply(lambda x: pd.Period(x, freq='M'))
    
    print("✅ Test Data loaded and Time Index simulated successfully.")
    
    
    # 2. Merge with the pre-processed Macro Data (df_macro)
    
    # df_macro was prepared in Block 4 and has TIME_INDEX as its actual index
    df_test_enriched = df_test.merge(
        df_macro, 
        left_on='TIME_INDEX', 
        right_index=True,  # Merge df_macro based on its index (TIME_INDEX)
        how='left'
    )
    
    print("✅ Test Data merged with Macro Data successfully.")
    print("\nEnriched Test Data Sample:")
    print(df_test_enriched[['SK_ID_CURR', 'TIME_INDEX', 'IPCA', 'SELIC']].head())
    print(f"\nFinal Enriched Test Shape: {df_test_enriched.shape}")

except FileNotFoundError:
    print(f"❌ ERROR: Test file not found at {RAW_TEST_FILE}. Check the Kaggle download path.")
except Exception as e:
    print(f"❌ An error occurred during test data loading or merging: {e}")

Attempting to load Test Data from: /content/drive/MyDrive/Project_01/data/raw/kaggle/home-credit-default-risk/application_test.csv
✅ Test Data loaded and Time Index simulated successfully.
✅ Test Data merged with Macro Data successfully.

Enriched Test Data Sample:
   SK_ID_CURR TIME_INDEX  IPCA  SELIC
0      100001    2018-05  0.40  14.25
1      100005    2013-03   NaN    NaN
2      100013    2017-10  0.42  14.25
3      100028    2017-05  0.31  14.25
4      100038    2018-02  0.32  14.25

Final Enriched Test Shape: (48744, 124)


## Treat Missing values and saving processed datasets

In [21]:
# NaN Treatment: Group-wise Mean Imputation for Macro Features and Median/Mode for Micro Features
print("Starting NaN treatment with Group-wise Mean Imputation (Macro) and Median/Mode (Micro)...")

# Identify Columns
EXCLUDE_COLS = ['SK_ID_CURR', 'TARGET', 'TIME_INDEX'] 
macro_cols = ['SELIC', 'IPCA']
all_cols = df_train_enriched.columns.tolist() 

# Columns that are purely micro features (excluding IDs, TARGET, and Macro columns)
original_micro_cols = [col for col in all_cols if col not in EXCLUDE_COLS + macro_cols]

# Identify column types among the micro features
object_micro_cols = df_train_enriched[original_micro_cols].select_dtypes(include=['object']).columns.tolist()
numeric_micro_cols = [col for col in original_micro_cols if col not in object_micro_cols]


# Categorical Imputation (Micro Features)

print("Imputing categorical features with 'Missing'...")

# Impute Categorical in Train
for col in object_micro_cols:
    df_train_enriched[col] = df_train_enriched[col].fillna('Missing') 

# Impute Categorical in Test
for col in object_micro_cols:
    df_test_enriched[col] = df_test_enriched[col].fillna('Missing') 
    
print("✅ Categorical features imputed.")


# Group-Wise Mean Imputation (Macro Features)

print("Imputing macro features with Group-wise Mean (by Month)...")

# Create a temporary 'Month' column (required for Period Index grouping)
df_train_enriched['Month'] = df_train_enriched['TIME_INDEX'].apply(lambda x: x.month)
df_test_enriched['Month'] = df_test_enriched['TIME_INDEX'].apply(lambda x: x.month)

# Calculate the monthly average ONLY from the TRAINING set (to prevent data leakage)
monthly_mean_map = {}
for col in macro_cols:
    # Calculate the mean of the column grouped by 'Month'
    monthly_mean_map[col] = df_train_enriched.groupby('Month')[col].mean()

# Impute Train
for col in macro_cols:
    # If the value is NaN, replace it with the mean of the corresponding Month
    df_train_enriched[col] = df_train_enriched.apply(
        lambda row: monthly_mean_map[col].loc[row['Month']] if pd.isna(row[col]) else row[col],
        axis=1
    )

# Impute Test (Using the mean map calculated from TRAIN)
for col in macro_cols:
    # If the value is NaN, replace it with the mean of the corresponding Month from the TRAIN map
    df_test_enriched[col] = df_test_enriched.apply(
        lambda row: monthly_mean_map[col].loc[row['Month']] if pd.isna(row[col]) else row[col],
        axis=1
    )

# Drop the temporary 'Month' column
df_train_enriched.drop('Month', axis=1, inplace=True)
df_test_enriched.drop('Month', axis=1, inplace=True)

print("✅ Macro features imputed with Group-wise Mean.")


# Median Imputation (Original Numerical Micro Features)

print("Imputing numerical micro features with Median...")

# Impute Train
# Calculate the median ONLY on the training set
train_medians = df_train_enriched[numeric_micro_cols].median()
df_train_enriched[numeric_micro_cols] = df_train_enriched[numeric_micro_cols].fillna(train_medians)

# Impute Test (Using the medians calculated from TRAIN)
df_test_enriched[numeric_micro_cols] = df_test_enriched[numeric_micro_cols].fillna(train_medians) 


# Final Verification and Save

print(f"\nFinal Check - Train Remaining NaNs: {df_train_enriched.isnull().sum().sum()}")
print(f"Final Check - Test Remaining NaNs: {df_test_enriched.isnull().sum().sum()}")


DATA_DIR_PROCESSED = os.path.join(DRIVE_BASE_PATH, 'data', 'processed')
if not os.path.exists(DATA_DIR_PROCESSED):
    os.makedirs(DATA_DIR_PROCESSED)

TRAIN_PROCESSED_FILE = os.path.join(DATA_DIR_PROCESSED, 'train_enriched.csv')
TEST_PROCESSED_FILE = os.path.join(DATA_DIR_PROCESSED, 'test_enriched.csv')

print("\nSaving treated and enriched datasets...")

try:
    df_train_enriched.to_csv(TRAIN_PROCESSED_FILE, index=False)
    print(f"✅ Saved treated training data to: {TRAIN_PROCESSED_FILE}")

    df_test_enriched.to_csv(TEST_PROCESSED_FILE, index=False)
    print(f"✅ Saved treated testing data to: {TEST_PROCESSED_FILE}")

except Exception as e:
    print(f"❌ ERROR saving files: {e}")

Starting NaN treatment with Group-wise Mean Imputation (Macro) and Median/Mode (Micro)...


Imputing categorical features with 'Missing'...
✅ Categorical features imputed.
Imputing macro features with Group-wise Mean (by Month)...
✅ Macro features imputed with Group-wise Mean.
Imputing numerical micro features with Median...

Final Check - Train Remaining NaNs: 0
Final Check - Test Remaining NaNs: 0

Saving treated and enriched datasets...
✅ Saved treated training data to: /content/drive/MyDrive/Project_01/data/processed/train_enriched.csv
✅ Saved treated testing data to: /content/drive/MyDrive/Project_01/data/processed/test_enriched.csv
